In [4]:
import requests
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# Load the variables from .env
load_dotenv()

# Access your API key
api_key = os.getenv("api_key")

city = "New York"
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"

response = requests.get(url)
data = response.json() 

print(data)

{'coord': {'lon': -74.006, 'lat': 40.7143}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 6.07, 'feels_like': 4.11, 'temp_min': 3.39, 'temp_max': 7.19, 'pressure': 1022, 'humidity': 61, 'sea_level': 1022, 'grnd_level': 1021}, 'visibility': 10000, 'wind': {'speed': 2.57, 'deg': 170}, 'clouds': {'all': 0}, 'dt': 1763682040, 'sys': {'type': 2, 'id': 2037026, 'country': 'US', 'sunrise': 1763639345, 'sunset': 1763674481}, 'timezone': -18000, 'id': 5128581, 'name': 'New York', 'cod': 200}


In [9]:
lat, lon = 40.6461, -111.4980
date = datetime(2024, 12, 1)
timestamp = int(date.timestamp())

url = f"https://pro.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&appid={api_key}&units=metric"

response = requests.get(url)
print(response.status_code)
print(response.text)



401
{"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}


In [6]:
# Locations: name → (lat, lon)
LOCATIONS = {
    "Park City": (40.6461, -111.4980),
    "Big Cottonwood": (40.6094, -111.7102)
}

# Define the date ranges for the winters you want
WINTERS = [
    (datetime(2022, 12, 1), datetime(2023, 2, 28)),
    (datetime(2023, 12, 1), datetime(2024, 2, 29)),
    (datetime(2024, 12, 1), datetime(2025, 2, 28)),
]

def get_date_range(start: datetime, end: datetime):
    """Yield each date from start to end, inclusive."""
    current = start
    while current <= end:
        yield current
        current += timedelta(days=1)

def get_weather_for_day(lat, lon, date):
    """Call OpenWeatherMap One Call API (historical) for a specific day."""
    timestamp = int(date.timestamp())
    url = (
        f"https://pro.openweathermap.org/data/2.5/onecall/timemachine"
        f"?lat={lat}&lon={lon}&dt={timestamp}&appid={api_key}&units=metric"
    )

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error for {date} at {lat},{lon}: {response.status_code}, {response.text}")
        return None
    return response.json()

def extract_weather_rows(data, location_name, date):
    """Convert API response into a list of daily/hourly rows."""
    rows = []
    # Use hourly data
    for hour in data.get("hourly", []):
        # Precipitation might be split into rain / snow
        precipitation = 0.0
        snowfall = 0.0

        if "rain" in hour:
            precipitation += hour["rain"].get("1h", 0)
        if "snow" in hour:
            snowfall += hour["snow"].get("1h", 0)
            precipitation += hour["snow"].get("1h", 0)

        rows.append({
            "location": location_name,
            "date": date.date(),
            "timestamp": hour["dt"],
            "temp": hour.get("temp"),
            "feels_like": hour.get("feels_like"),
            "pressure": hour.get("pressure"),
            "humidity": hour.get("humidity"),
            "wind_speed": hour.get("wind_speed"),
            "precipitation": precipitation,
            "snowfall": snowfall,
            "weather_main": hour["weather"][0].get("main"),
            "weather_desc": hour["weather"][0].get("description"),
        })
    return rows

def collect_weather_data():
    """Main function to collect data for all locations and winters."""
    all_data = []
    for loc_name, (lat, lon) in LOCATIONS.items():
        print(f"Collecting data for {loc_name}...")
        for (start, end) in WINTERS:
            for date in get_date_range(start, end):
                # Call API
                raw = get_weather_for_day(lat, lon, date)
                if raw:
                    rows = extract_weather_rows(raw, loc_name, date)
                    all_data.extend(rows)
                # Be kind to the API — avoid hammering too fast
                time.sleep(1) # adjust sleep to what your quota allows

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    return df

if __name__ == "__main__":
    df = collect_weather_data()
    print("Data collection complete. Sample:")
    print(df.head())

    # Save to CSV
    df.to_csv("winter_weather_bigcottonwood_parkcity.csv", index=False)
    print("Saved CSV.")

Error for 2022-12-01 00:00:00 at 40.6461,-111.498: 401, {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Error for 2022-12-02 00:00:00 at 40.6461,-111.498: 401, {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Error for 2022-12-03 00:00:00 at 40.6461,-111.498: 401, {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Error for 2022-12-04 00:00:00 at 40.6461,-111.498: 401, {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Error for 2022-12-05 00:00:00 at 40.6461,-111.498: 401, {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}
Error for 2022-12-06 00:00:00 at 40.6461,-111.498: 401, {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}


KeyboardInterrupt: 